In [35]:
import pandas as pd
import numpy as np
import rasterio
from rasterio.plot import show
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import seaborn as sns

# ------------------------
# 1. FAOSTAT Crop Yield Data
# ------------------------

In [36]:
# Load the datasets
faostat_df = pd.read_csv("1_FAOSTAT_data_en_5-13-2025.csv")

In [37]:
# Inspect the dataset
print("FAOSTAT Initial Info:")
print(faostat_df.info())
print(faostat_df.head())

FAOSTAT Initial Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5292 entries, 0 to 5291
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Domain Code       5292 non-null   object 
 1   Domain            5292 non-null   object 
 2   Area Code (M49)   5292 non-null   int64  
 3   Area              5292 non-null   object 
 4   Element Code      5292 non-null   int64  
 5   Element           5292 non-null   object 
 6   Item Code (CPC)   5292 non-null   float64
 7   Item              5292 non-null   object 
 8   Year Code         5292 non-null   int64  
 9   Year              5292 non-null   int64  
 10  Unit              5292 non-null   object 
 11  Value             5142 non-null   float64
 12  Flag              5292 non-null   object 
 13  Flag Description  5292 non-null   object 
 14  Note              102 non-null    object 
dtypes: float64(2), int64(4), object(9)
memory usage: 620.3+ KB
None
  D

In [38]:
# Select relevant columns (assuming Area, Item, Element, Year, Unit, Value are key)
faostat_df = faostat_df[['Area', 'Item', 'Element', 'Year', 'Unit', 'Value']]

In [39]:
# Filter for Ethiopia and relevant elements (Yield, Area harvested, Production)
faostat_df = faostat_df[faostat_df['Area'] == 'Ethiopia']
elements = ['Yield', 'Area harvested', 'Production']
faostat_df = faostat_df[faostat_df['Element'].isin(elements)]

In [40]:
# Check for missing values
print("FAOSTAT Missing Values:")
print(faostat_df.isnull().sum())

FAOSTAT Missing Values:
Area         0
Item         0
Element      0
Year         0
Unit         0
Value      150
dtype: int64


In [41]:
# Drop rows with missing values in 'Value' (if any)
faostat_df = faostat_df.dropna(subset=['Value'])

In [42]:
# Pivot the data to have Yield, Area harvested, and Production as columns
faostat_pivoted = faostat_df.pivot_table(
    index=['Area', 'Item', 'Year'],
    columns='Element',
    values='Value',
    aggfunc='first'
).reset_index()

In [43]:
# Rename columns for clarity
faostat_pivoted.columns = ['Area', 'Crop', 'Year', 'Area_Harvested', 'Production', 'Yield']

In [44]:
# Verify units (Yield: hg/ha, Area: ha, Production: tonnes)
print("FAOSTAT Units:")
print(faostat_df[['Element', 'Unit']].drop_duplicates())

FAOSTAT Units:
          Element   Unit
0  Area harvested     ha
1           Yield  kg/ha
2      Production      t


In [45]:
# Save cleaned FAOSTAT dataset
# faostat_pivoted.to_csv("Cleaned_FAOSTAT.csv", index=False)
print("FAOSTAT Cleaned Dataset Saved as 'Cleaned_FAOSTAT.csv'")

FAOSTAT Cleaned Dataset Saved as 'Cleaned_FAOSTAT.csv'


# ------------------------
# 2. NDVI Data
# ------------------------

In [46]:
# Load the datasets
ndvi_df = pd.read_csv("2_Ethiopia_NDVI_Monthly.csv")

In [47]:
# Inspect the dataset
print("NDVI Initial Info:")
print(ndvi_df.info())
print(ndvi_df.head())

NDVI Initial Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 277 entries, 0 to 276
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   system:index  277 non-null    int64  
 1   NDVI          277 non-null    float64
 2   date          277 non-null    object 
 3   .geo          277 non-null    object 
dtypes: float64(1), int64(1), object(2)
memory usage: 8.8+ KB
None
   system:index      NDVI     date                                    .geo
0             0  0.263435  2000-02  {"type":"MultiPoint","coordinates":[]}
1             1  0.258993  2000-03  {"type":"MultiPoint","coordinates":[]}
2             2  0.312295  2000-04  {"type":"MultiPoint","coordinates":[]}
3             3  0.437923  2000-05  {"type":"MultiPoint","coordinates":[]}
4             4  0.399441  2000-06  {"type":"MultiPoint","coordinates":[]}


In [48]:
# Drop irrelevant '.geo' column
ndvi_df = ndvi_df.drop(columns=['.geo', 'system:index'])

In [49]:
# Convert 'date' to datetime
ndvi_df['date'] = pd.to_datetime(ndvi_df['date'], format='%Y-%m')

In [50]:
# Validate NDVI values (should be between -1 and 1)
invalid_ndvi = ndvi_df[(ndvi_df['NDVI'] < -1) | (ndvi_df['NDVI'] > 1)]
if not invalid_ndvi.empty:
    print("Invalid NDVI values found:", invalid_ndvi)
else:
    print("All NDVI values are within valid range (-1 to 1).")

All NDVI values are within valid range (-1 to 1).


In [51]:
# Check for missing values
print("NDVI Missing Values:")
print(ndvi_df.isnull().sum())

NDVI Missing Values:
NDVI    0
date    0
dtype: int64


In [52]:
# Drop rows with missing NDVI values (if any)
ndvi_df = ndvi_df.dropna(subset=['NDVI'])

In [53]:
# Extract year and month for potential aggregation
ndvi_df['Year'] = ndvi_df['date'].dt.year
ndvi_df['Month'] = ndvi_df['date'].dt.month

# Aggregate NDVI to yearly averages (or seasonal, depending on crop cycles)
# For now, we'll compute yearly averages; adjust for seasonal if needed
ndvi_yearly = ndvi_df.groupby('Year')['NDVI'].mean().reset_index()

In [54]:
# Save cleaned NDVI dataset
# ndvi_yearly.to_csv("Cleaned_NDVI.csv", index=False)
print("NDVI Cleaned Dataset Saved as 'Cleaned_NDVI.csv'")

NDVI Cleaned Dataset Saved as 'Cleaned_NDVI.csv'


In [55]:
# Preview cleaned datasets
print("\nCleaned FAOSTAT Preview:")
print(faostat_pivoted.head())
print("\nCleaned NDVI Preview:")
print(ndvi_yearly.head())


Cleaned FAOSTAT Preview:
       Area                                               Crop  Year  \
0  Ethiopia  Anise, badian, coriander, cumin, caraway, fenn...  2000   
1  Ethiopia  Anise, badian, coriander, cumin, caraway, fenn...  2001   
2  Ethiopia  Anise, badian, coriander, cumin, caraway, fenn...  2002   
3  Ethiopia  Anise, badian, coriander, cumin, caraway, fenn...  2003   
4  Ethiopia  Anise, badian, coriander, cumin, caraway, fenn...  2004   

   Area_Harvested  Production  Yield  
0           800.0       500.0  625.0  
1           960.0       600.0  625.0  
2          1120.0       700.0  625.0  
3          1280.0       800.0  625.0  
4          1440.0       900.0  625.0  

Cleaned NDVI Preview:
   Year      NDVI
0  2000  0.400037
1  2001  0.401398
2  2002  0.400534
3  2003  0.405961
4  2004  0.405473


# ------------------------
# 3. NASA POWER Weather Data
# ------------------------

# ------------------------
### 3.1 Solar Radiation
# ------------------------

In [56]:
# Load the dataset
nasa_df = pd.read_csv('3_POWER_Regional_Monthly_2000_2023 (2).csv', skiprows=9)

In [57]:
# Inspect the dataset
print("NASA Power Initial Info:")
print(nasa_df.info())
print(nasa_df.head())

NASA Power Initial Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2160 entries, 0 to 2159
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   PARAMETER  2160 non-null   object 
 1   YEAR       2160 non-null   int64  
 2   LAT        2160 non-null   float64
 3   LON        2160 non-null   float64
 4   JAN        2160 non-null   float64
 5   FEB        2160 non-null   float64
 6   MAR        2160 non-null   float64
 7   APR        2160 non-null   float64
 8   MAY        2160 non-null   float64
 9   JUN        2160 non-null   float64
 10  JUL        2160 non-null   float64
 11  AUG        2160 non-null   float64
 12  SEP        2160 non-null   float64
 13  OCT        2160 non-null   float64
 14  NOV        2160 non-null   float64
 15  DEC        2160 non-null   float64
 16  ANN        2160 non-null   float64
dtypes: float64(15), int64(1), object(1)
memory usage: 287.0+ KB
None
           PARAMETER  YEAR   LAT   LON

In [58]:
# Check if PARAMETER is uniform (all ALLSKY_SFC_SW_DWN)
if nasa_df['PARAMETER'].nunique() == 1:
    nasa_df = nasa_df.drop(columns=['PARAMETER'])

In [59]:
# Replace -999 with NaN for missing data
nasa_df = nasa_df.replace(-999, np.nan)

In [60]:
# Check for missing values
print("NASA Power Missing Values:")
print(nasa_df.isnull().sum())

NASA Power Missing Values:
YEAR    0
LAT     0
LON     0
JAN     0
FEB     0
MAR     0
APR     0
MAY     0
JUN     0
JUL     0
AUG     0
SEP     0
OCT     0
NOV     0
DEC     0
ANN     0
dtype: int64


In [61]:
# Drop rows with NaN in all monthly columns (JAN-DEC)
monthly_cols = ['JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN', 'JUL', 'AUG', 'SEP', 'OCT', 'NOV', 'DEC']
nasa_df = nasa_df.dropna(subset=monthly_cols, how='all')

# Melt monthly columns into long format
nasa_melted = pd.melt(
    nasa_df,
    id_vars=['YEAR', 'LAT', 'LON'],
    value_vars=monthly_cols,
    var_name='Month',
    value_name='Solar_Radiation'
)

# Map month abbreviations to numbers
month_map = {
    'JAN': 1, 'FEB': 2, 'MAR': 3, 'APR': 4, 'MAY': 5, 'JUN': 6,
    'JUL': 7, 'AUG': 8, 'SEP': 9, 'OCT': 10, 'NOV': 11, 'DEC': 12
}
nasa_melted['Month'] = nasa_melted['Month'].map(month_map)

In [62]:
# Create a datetime column
nasa_melted['Date'] = pd.to_datetime(nasa_melted[['YEAR', 'Month']].assign(DAY=1))

In [63]:
# Drop rows with NaN in Solar_Radiation
nasa_melted = nasa_melted.dropna(subset=['Solar_Radiation'])

In [64]:
# Aggregate spatially (average across LAT/LON) for national-level data
nasa_agg = nasa_melted.groupby(['Date', 'YEAR'])['Solar_Radiation'].mean().reset_index()

# Compute yearly averages for alignment with FAOSTAT
nasa_yearly = nasa_agg.groupby('YEAR')['Solar_Radiation'].mean().reset_index()

In [65]:
# Save cleaned datasets
nasa_melted.to_csv("Cleaned_NASA_Power_SolarRadiation_monthly.csv", index=False)
nasa_yearly.to_csv("Cleaned_NASA_Power_SolarRadiation_cleaned_yearly.csv", index=False)
print("NASA Power Cleaned Datasets Saved as 'Cleaned_NASA_Power_SolarRadiation_monthly.csv' and 'Cleaned_NASA_Power_SolarRadiation_cleaned_yearly.csv'")

# Preview cleaned datasets
print("\nCleaned NASA Power Monthly Preview:")
print(nasa_melted.head())
print("\nCleaned NASA Power Yearly Preview:")
print(nasa_yearly.head())

NASA Power Cleaned Datasets Saved as 'Cleaned_NASA_Power_SolarRadiation_monthly.csv' and 'Cleaned_NASA_Power_SolarRadiation_cleaned_yearly.csv'

Cleaned NASA Power Monthly Preview:
   YEAR   LAT   LON  Month  Solar_Radiation       Date
0  2000  10.5  33.5      1            21.93 2000-01-01
1  2000  10.5  34.5      1            21.52 2000-01-01
2  2000  10.5  35.5      1            21.68 2000-01-01
3  2000  10.5  36.5      1            21.49 2000-01-01
4  2000  10.5  37.5      1            22.38 2000-01-01

Cleaned NASA Power Yearly Preview:
   YEAR  Solar_Radiation
0  2000        21.225880
1  2001        20.336213
2  2002        20.732056
3  2003        21.020926
4  2004        21.042417
